In [3]:
import pandas as pd
import altair as alt
import numpy as np

## Analisis de Previo - contexto

In [5]:
def extrae_drive(url):
    edit = '/edit#gid='
    export ='/export?format=xlsx&gid='
    url = (url).replace(edit, export)
    return url

### Ceses y Rotacion
* Los *ceses* son la cantidad de colaboradores que dejan de trabajar en la empresa en un determinado mes
* La *rotacion* es el % entre los colaboradores cesados y el total de colaboradores
* Saber cuales son los ceses y su comportamiento ayuda a tener una vision general, pero esto no es comparable entre areas, zonas o cargos.
    * Por qué? Pues un cargo puede tener 10 ceses pero como tiene en total 100 colaboradores su rotacion solo le impacta un 10%; sin embargo si un cargo tiene 5 ceses pero su total de colaboradores es de 10, su rotacion sera 50%  es decir mas impactante
* Debido a esto es importante analizar el comportamiento tomando en cuenta los Ceses y la rotacion que generan

In [161]:
ceses = extrae_drive('https://docs.google.com/spreadsheets/d/1bO_T0lAQgjcpjFFNVAwiXd2obwRjJS3K/edit#gid=2050623625')
ceses = pd.read_excel(
    ceses,
    usecols='E,I,O,P,S,T,W,X',
    dtype = {'CODIGO':str}
).rename(
    columns = {
        'Total o Temprana':'TIEMPO_CESE',
        'Producto':'PRODUCTO',
        'TIPO':'TIPO_CESE',
        'MOTIVO PRIMARIO':'MOTIVO_CESE'
    }
)

* Debido a un hecho extraordinario como una mudanza de una provincia a la capital los ceses del 2017 son atipicos, por lo se consideraran a partir del 2018
* Ademas el resto de bases parten desde este anio
* Es necesario transformar a inicio de mes la fecha de cese e ingreso porque es el intervalo que usare para comparar las temporalidades de otras bases

In [7]:
ceses.describe()

C:\Users\Anna\AppData\Local\Temp/ipykernel_6848/2583188236.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  ceses.describe()


,CODIGO,CARGO,INGRESO,CESE,TIEMPO_CESE,PRODUCTO,TIPO_CESE,MOTIVO_CESE
count,3466,3466,3466,3466,3466,3466,3466,3466
unique,3446,11,537,988,2,2,2,28
top,A86AAAAAA3,VENTA GRUPAL,2018-06-04 00:00:00,2020-01-02 00:00:00,-,GRUPAL,VOLUNTARIA,Mejor Oferta De Trabajo
freq,2,1984,61,38,2837,2245,2338,583
first,NaN,NaN,1999-10-11 00:00:00,2017-01-02 00:00:00,NaN,NaN,NaN,NaN
last,NaN,NaN,2021-11-19 00:00:00,2021-12-31 00:00:00,NaN,NaN,NaN,NaN


In [162]:
# Tranformando y filtrando lo necesario para el analisis
ceses = ceses.query('CESE >= "2018-01-01"')
ceses['CESE'] = ceses['CESE'].to_numpy().astype('datetime64[M]')
ceses['INGRESO'] = ceses['INGRESO'].to_numpy().astype('datetime64[M]')

* Analizo los ceses separando el producto por ser una variable que genera mucha diferencia en el negocio
* La rotacion en grupal en mas alta que en individual en numero de colaboradores cesados

In [9]:
ceses_prod = ceses.groupby(
    by=['PRODUCTO','CESE'],
    as_index = False
    ).agg(
        cesados = ('CODIGO', 'count'))
# plot por producto
barra = alt.Chart(
    ceses_prod
).mark_bar(
    color = '#04328C'
).encode(
    x = alt.X(
        'year(CESE):N',
        title = None),
    y = alt.Y(
        'sum(cesados):Q',
        title = None,
        axis = None)
).properties(
    width = 300,
    height = 200
)
texto = barra.mark_text(
    align = 'center',
    baseline ='top',
    dy = 5,
    color = '#F2F2F2',
    fontSize = 13,
    fontWeight = 'bold'
).encode(
    text = alt.Text('sum(cesados)')
)
# plot barra  y texto
(barra+texto).facet('PRODUCTO').properties(
    title = 'Total de Ceses por Producto Acumulado por Año'
).configure_view(
    strokeWidth = 0 #quita el borde
).configure_axis(
    labelFontSize = 15,
    labelAngle = 0
).configure_title(
    fontSize = 18,
    color = '#033E8C'
)


alt.FacetChart(...)

* Analizo el comportamineto de los ceses segun los cargos en cada tipo de producto
* Sebe calaramente que la fuerza de ventas (a diferecnia de los otros cargos) es la que genera mayor cantidad de ceses, esto podria deberse a que tambien son los cargos con mayor cantidad de personas activas, para esto sera necesaria analizar la rotacion por cargo

In [40]:
ceses_cargo = ceses.groupby(
    by=['PRODUCTO', 'CARGO', 'CESE'],
    as_index = False
    ).agg(
        cesados = ('CODIGO', 'count')
    )

alt.Chart(
    ceses_cargo
).mark_bar().encode(
    x = alt.X(
        'year(CESE):N',
        title = None),
    y = alt.Y(
        'sum(cesados):Q',
        title = None),
    color = alt.Color('CARGO:O'),
    column = alt.Column(
        'PRODUCTO',
        title = None)
).properties(
    width = 300,
    height = 200,
    title = 'Total ceses por Cargo y Producto Acumulado por Año'
).configure_view(
    strokeWidth = 0 #quita el borde
).configure_axis(
    labelFontSize = 15,
    labelAngle = 0
).configure_title(
    fontSize = 18,
    color = '#033E8C'
)

alt.Chart(...)

#### Rotacion
* Para analizar la rotacion necesitare unir mi base de planilla con la de ceses para sacar el % de ceses entre el total de colaboradores
* Esto me ayudara a saber si la hipotesis de que los cargos de Venta Grupal e Individual son los mas importantes de analizar por tener la mayor cantidad de colaboradores cesados y una alta rotacion

In [11]:
planilla = extrae_drive('https://docs.google.com/spreadsheets/d/1L094YBwj3BvobbXb4t3ahzpmL6gBSxgs/edit#gid=648976010')
planilla = pd.read_excel(
    planilla,
    usecols='A,B,D:H',
    dtype={'COD':str}
).rename(
    columns={
        'COD':'CODIGO',
        'REGION/DPTO':'REGION',
        'DIVISION/GC':'DIVISION'

    }
)

* Uno la base de ceses con la de planilla agrupada hasta cargo para visualizar la rotacion por mes
* Para este caso lo que quiero comprar es la rotacion anual, la cual esta compuesta por la suma de las rotaciones mensuales, para ello creo la columna year

In [12]:
head_cargo = planilla.groupby(
    by=['PRODUCTO', 'CARGO', 'MES'],
    as_index = False
    ).agg(
        real = ('CODIGO', 'count')
    )
## Join ceses y planilla
rot_cargo = pd.merge(
    head_cargo,
    ceses_cargo,
    left_on=['MES', 'PRODUCTO', 'CARGO'],
    right_on=['CESE', 'PRODUCTO', 'CARGO'])
# hallo la rotacion cesados/real
rot_cargo['rotacion'] = np.where(
    rot_cargo['real'] == 0,
    0,
    ((rot_cargo['cesados'] / rot_cargo['real'])*100).round(1))
    
rot_cargo['year'] = rot_cargo['MES'].to_numpy().astype('datetime64[Y]')

* Grafico la rotacion de los cargos del Producto Grupal e Individual agrupado de manera anual
* Se muestra que los cargos de Ventas Individual y Ventas Grupal tuvieron una alta rotacion contante en estos ultimos a;os
* Hasta el momento concluimos que el analisis se debe hacer en esta primera etapa a los cargos de ventas, ya se tendria un impacto mayor a los otros cargos

In [13]:
#### Grafico Grupal
rot_cargo_g = rot_cargo.query('PRODUCTO == "GRUPAL"'
    ).groupby(
        by = ['year', 'PRODUCTO', 'CARGO'],
        as_index = False
    ).agg(
        rotacion = ('rotacion','sum')
    )
## Grafico
barra_g = alt.Chart(
    rot_cargo_g
).mark_bar(
    color = '#04328C'
).encode(
    x = alt.Y(
        'year(year):N',
        title = None),
    y = alt.X(
        'rotacion:Q',
        title = None,
        axis = None)
).properties(
    width = 200,
    height = 200
)

texto_g = barra_g.mark_text(
    align = 'center',
    baseline ='top',
    dy = 3,
    color = '#F2F2F2',
    fontSize = 13,
    fontWeight = 'bold'
).encode(
    text = alt.Text('rotacion')
)

gru = (barra_g + texto_g).facet(
    'CARGO',
    columns = 6
).properties(
    title = '%Rotacion Anual por Cargo de Grupal')

#### Grafico Individual
rot_cargo_i = rot_cargo.query('PRODUCTO == "INDIVIDUAL"'
    ).groupby(
        by = ['year', 'PRODUCTO', 'CARGO'],
        as_index = False
    ).agg(
        rotacion = ('rotacion','sum')
    )
barra_i = alt.Chart(
    rot_cargo_i
).mark_bar(
    color = '#04328C'
).encode(
    x = alt.Y(
        'year(year):N',
        title = None),
    y = alt.X(
        'rotacion:Q',
        title = None,
        axis = None)
).properties(
    width = 200,
    height = 200
)

texto_i = barra_i.mark_text(
    align = 'center',
    baseline ='top',
    dy = 3,
    color = '#F2F2F2',
    fontSize = 13,
    fontWeight = 'bold'
).encode(
    text = alt.Text('rotacion')
)

ind = (barra_i + texto_i).facet(
    'CARGO',
    columns = 5
).properties(
    title = '%Rotacion Anual por Cargo de Individual')

##### Uniendo ambos graficos
alt.vconcat(gru, ind).configure_view(
    stroke = '#04B2D9',
    strokeWidth = 0.3 
).configure_axis(
    labelFontSize = 15,
    labelAngle = 0
).configure_title(
    fontSize = 18,
    color = '#04328C'
)

alt.VConcatChart(...)

#### Analizando la antiguedad de los cesados Ventas Individual y Grupal
* En la empresa se toma en cuenta como ceses tempranos a los que cesan por cualquier motivo antes de los 6 meses
* Seria importante ver el comportamiento en antiguedad de los ceses para saber en que mes cesan mas y ne que mes los activos se empiezan a mantener
* Necesitamos hallar la antiguedad, fecha de CESE - fecha INGRESO

In [163]:
ceses = ceses.query('CARGO == "VENTA INDIVIDUAL" | CARGO == "VENTA GRUPAL"')
ceses['antiguedad_meses'] = (((ceses['CESE'] - ceses['INGRESO']).dt.days)/30).round(0)

* Se ve una diferencia en el comportamiento en la antiguedad de ceses voluntarios vs ceses involuntarios
* Los colores mas claros reflejan la clasificacion que la empresa le da a los ceses menores de 6 meses como tempranos
* Los ceses involuntarios tiene un comportamiento diferente durante los 9 primeros meses respecto al resto de meses mayores o iguales a 12
* En los ceses voluntarios el comportamiento diferente se da solo los en 3 primeros meses, del mes 6 adelante los ceses disminuyen con el tiempo
* Debido a este comportamiento diferente se debe hacer una analisis y prediccion para cada tipo de cese. **Considerando que los ceses involuntarios se deben despidos por diferente motivos, este analisis repercutiria en otro proyecto que vaya mas alineado al tema de contrataciones, ya que los colaboradores contratados no cumplen con la expectativa de la empresa.**
* Por lo tanto, para este proyecto se tomaran solo los ceses voluntarios, **al parecer la clasificacion de antiguedad de cese tomada por la empresa (en el caso de ceses voluntarios) seria incorrecta, pues el cambio de comportamiento se da desde el mes 3**. Seria necesario analizar por Grupal e Individual para validar esta hipotesis

In [135]:
ceses_antg = ceses.groupby(
    by=['TIPO_CESE','antiguedad_meses'],
    as_index=False
).agg(
    cesados = ('CODIGO', 'count'))

# GRAFICANDO
alt.Chart(ceses_antg).mark_bar(
    color = '#04328C'
).encode(
    x = alt.X(
        'antiguedad_meses',
        bin = alt.BinParams(step=3),
        title = 'Antiguedad (meses)'),
    y = alt.Y(
        'cesados',
        title = 'Total Ceses'),
    color = alt.condition(
        alt.datum.antiguedad_meses < 6.0, 
        alt.value('#04B2D9'),
        alt.value('#04328C')),
    column = alt.Column(
        'TIPO_CESE',
        title = None)
).properties(
    title = 'Total de Ceses Segun Antiguedad por Producto',
    width = 500,
    height = 250
).configure_view(
    strokeWidth = 0
).configure_axis(
    labelFontSize = 15,
    labelAngle = 0
).configure_title(
    fontSize = 18,
    color = '#033E8C'
).resolve_scale(
    y='independent'
)

alt.Chart(...)

* Analizando solo el comportamiento de Ceses Voluntarios

In [164]:
ceses = ceses.query('TIPO_CESE == "VOLUNTARIA"')

* Para los Vendedores de Grupal se muestra un comportamiento de a mayor antiguedad menor cantidad de ceses, esto a partir del mes 3
* Sin embargo, para los Vendedores Individuales en el comportamiento es alparecer bimodal. **Desde le mes 3 la cantidad de ceses se reduce con el tiempo, pero desde el mes 12 empieza a subir nuevamente hasta el mes 21, donde recien empieza a tener un comportamiento un poco mas constante en el tiempo**
* Se podria tomar el rango de 3 a 19 meses para predecir un comportamiento determinado, debido a que si separamos este rango de meses en dos los peridos serian muy cortos para analizar el comportamiento de los colaboradores
* Se considerara para un analisis mejor en invidual el comportamiento para el cese apartir del mes 21

In [138]:
ceses_antg_g = ceses.groupby(
    by=['PRODUCTO','antiguedad_meses'],
    as_index=False
).agg(
    cesados = ('CODIGO', 'count')
).query('PRODUCTO == "GRUPAL"'
)

# GRAFICANDO grupal
barra_g = alt.Chart(ceses_antg_g).mark_bar(
    color = '#04328C'
).encode(
    x = alt.X(
        'antiguedad_meses',
        bin = alt.BinParams(step=3),
        title = 'Antiguedad (meses)'),
    y = alt.Y(
        'cesados',
        title = 'Total Ceses')
)
linea_g_6 = alt.Chart(
    pd.DataFrame({'antiguedad_meses': [3]})
).mark_rule(
    color = 'red',
    strokeWidth = 2
).encode(
    x = 'antiguedad_meses')

gru = (barra_g + linea_g_6
).properties(title = 'Total de Ceses Voluntarios Segun Antiguedad - Grupal',
    width = 600,
    height = 250
)
############
ceses_antg_i = ceses.groupby(
    by=['PRODUCTO','antiguedad_meses'],
    as_index=False
).agg(
    cesados = ('CODIGO', 'count')
).query('PRODUCTO == "INDIVIDUAL"'
)

# GRAFICANDO indiviudal
barra_i = alt.Chart(ceses_antg_i).mark_bar(
    color = '#04328C'
).encode(
    x = alt.X(
        'antiguedad_meses',
        bin = alt.BinParams(step=3),
        title = 'Antiguedad (meses)'),
    y = alt.Y(
        'cesados',
        title = 'Total Ceses')
)
linea_i_6 = alt.Chart(
    pd.DataFrame({'antiguedad_meses': [3]})
).mark_rule(
    color = 'red',
    strokeWidth = 2
).encode(
    x = 'antiguedad_meses')

linea_i_12 = alt.Chart(
    pd.DataFrame({'antiguedad_meses': [12]})
).mark_rule(
    color = 'red',
    strokeWidth = 2
).encode(
    x = 'antiguedad_meses')

linea_i_21 = alt.Chart(
    pd.DataFrame({'antiguedad_meses': [21]})
).mark_rule(
    color = 'red',
    strokeWidth = 2
).encode(
    x = 'antiguedad_meses')

ind = (barra_i + linea_i_6  + linea_i_21).properties(
    title = 'Total de Ceses Voluntarios Segun Antiguedad - Individual',
    width = 600,
    height = 250
)
### juntando graficos
alt.vconcat(gru,ind
).configure_view(
    strokeWidth = 0
).configure_axis(
    labelFontSize = 15,
    labelAngle = 0
).configure_title(
    fontSize = 18,
    color = '#033E8C'
).resolve_scale(
    y='independent'
)

alt.VConcatChart(...)

### Motivos de Cese
* La empresa cuenta con una clasificacion de motivos de cese, pese a que estos motivos podrian ser un poco subjetivos o falsos debido a que no hya forma de verificarlos al 100%, seria interesante ver los principales motivos segun el analisis que se hizo previamente

In [165]:
ceses_g = ceses.query('PRODUCTO == "GRUPAL" and antiguedad_meses >= 3')

In [248]:
motivo_cese = ceses_g.groupby(
    by = ['MOTIVO_CESE'],
    as_index = False
).agg(
    antg_prom = ('antiguedad_meses', 'median')
)

barra = alt.Chart(motivo_cese).mark_bar().encode(
    y = alt.Y(
        'MOTIVO_CESE',
        sort='x'),
    x = alt.X(
        'antg_prom',
        title = None,
        axis = None),
    color = alt.condition(
        alt.datum.MOTIVO_CESE == 'Falsas Expectativas' , 
        alt.value("orange"),
        alt.value("steelblue"))
)

texto = barra.mark_text(
    align = 'center',
    baseline ='bottom',
    dy = 7,
    dx = -15,
    color = '#F2F2F2',
    fontSize = 12,
    fontWeight = 'bold'
).encode(
    text = alt.Text('antg_prom',format =',.2r')
)

barra+texto

alt.LayerChart(...)

In [228]:
ceses_g.groupby(
    by = ['MOTIVO_CESE'],
    as_index = False
).agg(
    cesados = ('CODIGO', 'count'),
    antg_mean = ('antiguedad_meses', 'mean'),
    antg_median = ('antiguedad_meses', 'median')
).sort_values(by='antg_median', ascending = False)

,MOTIVO_CESE,cesados,antg_mean,antg_median,antg_dsv
11,Problemas Economicos,2,25.500000,25.5,26.162951
1,Cuidado y atencion de hijos,84,24.238095,22.5,17.137023
2,Desacuerdo En Cambios Laborales,21,24.809524,21.0,20.991949
7,Mejor Oferta De Trabajo,240,23.408333,20.0,16.030043
5,Liderazgo,56,20.750000,18.0,14.671246
0,Cambio De Residencia,62,19.741935,17.5,14.562399
8,Negocio Propio,88,22.863636,17.0,17.584072
13,Situaciones Personales y Familiares,51,17.215686,17.0,9.804721
6,Matrimonio,31,16.903226,16.0,10.988949
4,Falsas Expectativas,47,17.936170,15.0,12.348920


In [170]:
ceses_i = ceses.query('PRODUCTO == "INDIVIDUAL" and antiguedad_meses >= 3')

In [245]:
motivo_cese = ceses_i.groupby(
    by = ['MOTIVO_CESE'],
    as_index = False
).agg(
    antg_prom = ('antiguedad_meses', 'median')
)

barra = alt.Chart(motivo_cese).mark_bar().encode(
    y = alt.Y(
        'MOTIVO_CESE',
        sort='x'),
    x = alt.X(
        'antg_prom',
        title = None,
        axis = None),
    color = alt.condition(
        alt.datum.antg_prom < 21 , 
        alt.value("orange"),
        alt.value("steelblue"))
)

texto = barra.mark_text(
    align = 'center',
    baseline ='bottom',
    dy = 7,
    dx = -15,
    color = '#F2F2F2',
    fontSize = 12,
    fontWeight = 'bold'
).encode(
    text = alt.Text('antg_prom')
)

barra+texto

alt.LayerChart(...)

* %Rotoacion por casgos de Individual


planilla = extrae_drive('https://docs.google.com/spreadsheets/d/1L094YBwj3BvobbXb4t3ahzpmL6gBSxgs/edit#gid=648976010')
entrevistas = extrae_drive('https://docs.google.com/spreadsheets/d/1bO_T0lAQgjcpjFFNVAwiXd2obwRjJS3K/edit#gid=2050623625')
programa_mejora = extrae_drive('https://docs.google.com/spreadsheets/d/1CKFlLpx248yvZS0INAawAA5e8NC2qdvV/edit#gid=1510069928')
memos = extrae_drive('https://docs.google.com/spreadsheets/d/1Q2pvICQWLAh8Rvou9FvH085KULXioDVK/edit#gid=238204385')
vacaciones = extrae_drive('https://docs.google.com/spreadsheets/d/1riJOPN5KUwV1962wO6mjPL7PMgL2v3_s/edit#gid=800276956')
descanso_medico = extrae_drive('https://docs.google.com/spreadsheets/d/1H_oENDjS0FyDNVmSg1199PrE6s8q6z7-/edit#gid=1138295584')
bonos_venta_i = extrae_drive('https://docs.google.com/spreadsheets/d/1Vjh5S86c1r60L-CnuQB_m6JZPy223Z4p/edit#gid=100516943834575165411')
bonos_venta_g = extrae_drive('https://docs.google.com/spreadsheets/d/1Vjh5S86c1r60L-CnuQB_m6JZPy223Z4p/edit#gid=100516943834575165411')
desempeno = extrae_drive()
reconocimientos = extrae_drive()
hijxs = extrae_drive()
datos_personales = extrae_drive()
recategorizaciones = extrae_drive()
cobertura_ofi = extrae_drive()
antg_oficina = extrae_drive()
gptw_oficina = extrae_drive()